In [3]:
from llm_blocks import chat_utils

In [2]:
template = """
Do you know about the LeetCode problem {LeetCodeProblemName}? If you do, can you explain it to me?
Detail the problem description, constraints, and examples.
Outline the steps you would take to solve the problem on a high level.
Write the code for the solution.
Add a summary of the problem and solution.
"""

test_chain = chat_utils.GenericChain(template=template, stream=True, model_name="gpt-4")
response_generator = test_chain("Two Sum")
chat_utils.stream_to_console(response_generator)

Problem Description:
The Two Sum problem on LeetCode is a common coding interview question. The problem is described as follows:

Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target. You may assume that each input would have exactly one solution, and you may not use the same element twice. You can return the answer in any order.

Constraints:
- The number of elements in the array is between 2 and 10^4.
- The integers in the array are between -10^9 and 10^9.
- The target value is also an integer between -10^9 and 10^9.
- There is exactly one solution for each input.

Examples:
Example 1:
Input: nums = [2,7,11,15], target = 9
Output: [0,1]
Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].

Example 2:
Input: nums = [3,2,4], target = 6
Output: [1,2]

Example 3:
Input: nums = [3,3], target = 6
Output: [0,1]

High-Level Steps:
1. Initialize a dictionary/hashmap to store the numbers in the array and their indice